In [ ]:
from google.colab import files
import pandas as pd

In [ ]:
f = files.upload()

Saving Data_11-20-2020.csv to Data_11-20-2020 (1).csv


In [ ]:
raw_data = pd.read_csv(next(iter(f)))

#preview dataset
raw_data.head(5)

,UnitID,Institution Name,Total number of full-time first-time degree/certificate seeking undergraduates - financial aid cohort (SFA1819),Number of students in fall cohort (SFA1819),Number of students in full-year cohort (SFA1819),Number of full-time first-time undergraduates awarded any financial aid (SFA1819),Percent of full-time first-time undergraduates awarded any financial aid (SFA1819),Number of full-time first-time undergraduates awarded any loans to students or grant aid from federal state/local government or the institution (SFA1819),Percent of full-time first-time undergraduates awarded any loans to students or grant aid from federal state/local government or the institution (SFA1819),Number of full-time first-time undergraduates awarded federal state local or institutional grant aid (SFA1819),Percent of full-time first-time undergraduates awarded federal state local or institutional grant aid (SFA1819),Total amount of federal state local or institutional grant aid awarded to full-time first-time undergraduates (SFA1819),Average amount of federal state local or institutional grant aid awarded (SFA1819),Number of full-time first-time undergraduates awarded federal grant aid (SFA1819),Percent of full-time first-time undergraduates awarded federal grant aid (SFA1819),Total amount of federal grant aid awarded to full-time first-time undergraduates (SFA1819),Average amount of federal grant aid awarded to full-time first-time undergraduates (SFA1819),Number of full-time first-time undergraduates awarded Pell grants (SFA1819),Percent of full-time first-time undergraduates awarded Pell grants (SFA1819),Total amount of Pell grant aid awarded to full-time first-time undergraduates (SFA1819),Average amount of Pell grant aid awarded to full-time first-time undergraduates (SFA1819),Number of full-time first-time undergraduates awarded other federal grant aid (SFA1819),Percent of full-time first-time undergraduates awarded other federal grant aid (SFA1819),Total amount of other federal grant aid awarded to full-time first-time undergraduates (SFA1819),Average amount of other federal grant aid awarded to full-time first-time undergraduates (SFA1819),Number of full-time first-time undergraduates awarded state/local grant aid (SFA1819),Percent of full-time first-time undergraduates awarded state/local grant aid (SFA1819),Total amount of state/local grant aid awarded to full-time first-time undergraduates (SFA1819),Average amount of state/local grant aid awarded to full-time first-time undergraduates (SFA1819),Number of full-time first-time undergraduates awarded institutional grant aid (SFA1819),Percent of full-time first-time undergraduates awarded institutional grant aid (SFA1819),Total amount of institutional grant aid awarded to full-time first-time undergraduates (SFA1819),Average amount of institutional grant aid awarded to full-time first-time undergraduates (SFA1819),Number of full-time first-time undergraduates awarded student loans (SFA1819),Percent of full-time first-time undergraduates awarded student loans (SFA1819),Total amount of student loans awarded to full-time first-time undergraduates (SFA1819),Average amount of student loans awarded to full-time first-time undergraduates (SFA1819),Number of full-time first-time undergraduates awarded federal student loans (SFA1819),Percent of full-time first-time undergraduates awarded federal student loans (SFA1819),Total amount of federal student loans awarded to full-time first-time undergraduates (SFA1819),...,Average amount of federal state local or institutional grant aid awarded (SFA1516_RV),Number of full-time first-time undergraduates awarded federal grant aid (SFA1516_RV),Percent of full-time first-time undergraduates awarded federal grant aid (SFA1516_RV),Total amount of federal grant aid awarded to full-time first-time undergraduates (SFA1516_RV),Average amount of federal grant aid awarded to full-time first-time undergraduates (SFA1516_RV),Number of full-time first-t

In [ ]:
#get columns name and sort, but keep 'UnitID' and 'Insitution Name' at the front
col_names = raw_data.columns.tolist()
first_three_cols = col_names[:2]
other_cols = col_names[2:]
other_cols.sort()
col_names = first_three_cols + other_cols

In [ ]:
#sort dataframe by columns name 
data = raw_data[col_names]

#show sorted column names
data.columns

Index(['UnitID', 'Institution Name',
       'Average amount of Pell grant aid awarded to full-time first-time undergraduates (SFA1516_RV)',
       'Average amount of Pell grant aid awarded to full-time first-time undergraduates (SFA1617_RV)',
       'Average amount of Pell grant aid awarded to full-time first-time undergraduates (SFA1718_RV)',
       'Average amount of Pell grant aid awarded to full-time first-time undergraduates (SFA1819)',
       'Average amount of federal  state  local or institutional grant aid awarded (SFA1516_RV)',
       'Average amount of federal  state  local or institutional grant aid awarded (SFA1617_RV)',
       'Average amount of federal  state  local or institutional grant aid awarded (SFA1718_RV)',
       'Average amount of federal  state  local or institutional grant aid awarded (SFA1819)',
       ...
       'Total amount of state/local grant aid awarded to full-time first-time undergraduates (SFA1819)',
       'Total amount of student loans awarded to 

In [ ]:
#drop columns with all NaNs:
#https://stackoverflow.com/questions/47744312/delete-a-column-that-has-all-nans-in-pandas
data = data.dropna(how='all', axis=1)
data.columns

Index(['UnitID', 'Institution Name',
       'Average amount of Pell grant aid awarded to full-time first-time undergraduates (SFA1516_RV)',
       'Average amount of Pell grant aid awarded to full-time first-time undergraduates (SFA1617_RV)',
       'Average amount of Pell grant aid awarded to full-time first-time undergraduates (SFA1718_RV)',
       'Average amount of Pell grant aid awarded to full-time first-time undergraduates (SFA1819)',
       'Average amount of federal  state  local or institutional grant aid awarded (SFA1516_RV)',
       'Average amount of federal  state  local or institutional grant aid awarded (SFA1617_RV)',
       'Average amount of federal  state  local or institutional grant aid awarded (SFA1718_RV)',
       'Average amount of federal  state  local or institutional grant aid awarded (SFA1819)',
       ...
       'Total amount of state/local grant aid awarded to full-time first-time undergraduates (SFA1718_RV)',
       'Total amount of state/local grant aid 

In [ ]:
#update the variable, 'col_names', since we dropped columns with all NaN
col_names = data.columns

In [ ]:
#turn dataframe into list of lists
prepared_data = data.values.tolist()
university_numbers = 0
universities = {}
for i in range(len(prepared_data)):
    if prepared_data[i][1] not in universities:
        universities[prepared_data[i][1]] = 1
        university_numbers += 1

In [ ]:
#how many universities in the dataset
university_numbers

25

In [ ]:
#see how the universities in the dataset
for n, k in enumerate(universities.keys()):
    print(str(n)+".", k)

0. Indiana University-Bloomington
1. Michigan State University
2. Ohio State University-Main Campus
3. Pennsylvania State University-Main Campus
4. Purdue University-Main Campus
5. Texas A & M University-College Station
6. Texas State University
7. Texas Tech University
8. The University of Texas at Arlington
9. The University of Texas at Austin
10. The University of Texas at Dallas
11. The University of Texas at El Paso
12. The University of Texas at San Antonio
13. The University of Texas at Tyler
14. University of California-Berkeley
15. University of California-Los Angeles
16. University of California-San Diego
17. University of Houston
18. University of Illinois at Urbana-Champaign
19. University of Michigan-Ann Arbor
20. University of Minnesota-Twin Cities
21. University of North Carolina at Chapel Hill
22. University of North Texas
23. University of Washington-Seattle Campus
24. University of Wisconsin-Madison


In [ ]:
#tranform data into "columns=['University', 'Categories', 'Year', 'Values']"
transforming_data = []
years = {'SFA1516':2015, 'SFA1617':2016, 'SFA1718':2017, 'SFA1819':2018}

for col_num in range(3, len(col_names)):
    for i in range(university_numbers):
        new_row = []
        
        #get the institution name
        new_row.append(prepared_data[i][1])
        
        #get the category name
        new_row.append('Fiancial Aids')

        #get the category name and remove year tag
        for pos in range(len(col_names[col_num])-2):
            if col_names[col_num][pos:pos+3] == '(SF' or col_names[col_num][pos:pos+3] == '201':
                new_row.append(col_names[col_num][:pos-1])
                break
        
        #get the year from year tag
        for key in years.keys():
            if key in col_names[col_num]:
                new_row.append(years[key])
        
        #get the values
        new_row.append(prepared_data[i][col_num])
        
        transforming_data.append(new_row)

#preview a sample (a row) of transformed dataset
transforming_data[0]

['Indiana University-Bloomington',
 'Fiancial Aids',
 'Average amount of Pell grant aid awarded to full-time first-time undergraduates',
 2016,
 4528]

In [ ]:
#remove some ' awarded to full-time first-time undergraduates' in column names
n = len(transforming_data)

for i in range(n):
    category = transforming_data[i][1]
    for pos in range(len(category)):
        if category[pos:] == ' awarded to full-time first-time undergraduates':
            category = category[:pos]
    transforming_data[i][1] = category

In [ ]:
#form the data into dataframe and give column names
transformed_data = pd.DataFrame(transforming_data, columns=['University', 'Category', 'Sub-Category', 'Year', 'Values'])
transformed_data.head(5)

,University,Category,Sub-Category,Year,Values
0,Indiana University-Bloomington,Fiancial Aids,Average amount of Pell grant aid awarded to fu...,2016,4528.0
1,Michigan State University,Fiancial Aids,Average amount of Pell grant aid awarded to fu...,2016,4799.0
2,Ohio State University-Main Campus,Fiancial Aids,Average amount of Pell grant aid awarded to fu...,2016,4326.0
3,Pennsylvania State University-Main Campus,Fiancial Aids,Average amount of Pell grant aid awarded to fu...,2016,4340.0
4,Purdue University-Main Campus,Fiancial Aids,Average amount of Pell grant aid awarded to fu...,2016,4348.0


In [ ]:
#export dateframe to csv file:
#https://www.geeksforgeeks.org/saving-a-pandas-dataframe-as-a-csv/
transformed_data.to_csv('data.csv')